In [ ]:
import glob
import pymysql
import pandas as pd
from sqlalchemy import create_engine

from django.core.exceptions import ObjectDoesNotExist
from django.conf import settings

In [ ]:
dbc = settings.LEGACY_DB_KREMS_CONNECTION

In [ ]:
db_connection_str = f"mysql+pymysql://{dbc['USER']}:{dbc['PASSWORD']}@{dbc['HOST']}/{dbc['NAME']}"
db_connection = create_engine(db_connection_str)

# enrich places
* fetch krems_ids from 'orte__ortsId' / 'orte__aschachId'
* fetch matching uris from 'orte_verknuepfungen__ortsId' / 'orte_verknuepfungen__eintrag'

In [ ]:
query = f"SELECT ortsname, ortsId, aschachId FROM orte WHERE aschachId IS NOT NULL"

In [ ]:
df = pd.read_sql(query, con=db_connection)

In [ ]:
df.head()

In [ ]:
for i, row in df.iterrows():
    try:
        ort = Ort.objects.get(legacy_pk=row['aschachId'])
    except ObjectDoesNotExist:
        print(row['ortsname'], row['aschachId'])
        continue
    ort.donau_normid = row['ortsId']
    ort.save()

In [ ]:
good_ids = Ort.objects.exclude(donau_normid=None).values('donau_normid')

In [ ]:
good_ids = [x['donau_normid'] for x in good_ids]

In [ ]:
query = f"""
SELECT
    ortsId,
    eintrag,
    art
FROM
    orte_verknuepfungen
INNER JOIN
    verknuepfungsArt ON orte_verknuepfungen.verknuepfungsArtId = verknuepfungsArt.verknuepfungsArtId
WHERE
    art <> 'Karte/Map' and art <> 'Google Maps'
ORDER BY
    ortsId 
    
"""

In [ ]:
df = pd.read_sql(query, con=db_connection)

In [ ]:
df.head()

In [ ]:
for i, row in df.iterrows():
    try:
        ort = Ort.objects.get(donau_normid=row['ortsId'])
    except ObjectDoesNotExist:
        continue
    try:
        uri_str = row['eintrag'].split('\\')[0]
    except AttributeError:
        continue
    uri, _ = Uri.objects.get_or_create(
        uri=uri_str
    )
    uri.domain = row['art']
    uri.save()
    ort.normdata_url.add(uri)